In [1]:
import os
import sqlite3

import google.generativeai as genai
from polars import read_parquet
from qdrant_client import QdrantClient

df = read_parquet("../data/interim/srag_2019_2024.parquet")
DATA_COLS = {col: str(df[col].dtype) for col in df.columns}
df.head()

c:\Users\VDUART10\Desktop\Tests\test_indicium\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_COLS

{'NU_NOTIFIC': 'Int64',
 'DT_NOTIFIC': 'Float64',
 'SEM_NOT': 'Int64',
 'DT_SIN_PRI': 'Float64',
 'SEM_PRI': 'Int64',
 'SG_UF_NOT': 'Categorical',
 'ID_REGIONA': 'Categorical',
 'CO_REGIONA': 'Float64',
 'ID_MUNICIP': 'Categorical',
 'CO_MUN_NOT': 'Float64',
 'CS_SEXO': 'Categorical',
 'DT_NASC': 'Float64',
 'NU_IDADE_N': 'Int64',
 'TP_IDADE': 'Int64',
 'COD_IDADE': 'Int64',
 'CS_GESTANT': 'Int64',
 'CS_RACA': 'Float64',
 'CS_ETINIA': 'Categorical',
 'CS_ESCOL_N': 'Float64',
 'ID_PAIS': 'Categorical',
 'CO_PAIS': 'Float64',
 'SG_UF': 'Categorical',
 'ID_RG_RESI': 'Categorical',
 'CO_RG_RESI': 'Float64',
 'ID_MN_RESI': 'Categorical',
 'CO_MUN_RES': 'Float64',
 'CS_ZONA': 'Float64',
 'NOSOCOMIAL': 'Float64',
 'AVE_SUINO': 'Float64',
 'FEBRE': 'Float64',
 'TOSSE': 'Float64',
 'GARGANTA': 'Float64',
 'DISPNEIA': 'Float64',
 'DESC_RESP': 'Float64',
 'SATURACAO': 'Float64',
 'DIARREIA': 'Float64',
 'VOMITO': 'Float64',
 'OUTRO_SIN': 'Float64',
 'OUTRO_DES': 'Categorical',
 'FATOR_RISC': 'Flo

In [ ]:
class AIQueryAgent:
    """
    Um agente que recebe perguntas em linguagem natural,
    traduz para SQL, consulta um SQLite, sumariza os resultados
    e busca notícias relacionadas no Qdrant.
    """

    def __init__(
        self,
        sqlite_path: str,
        qdrant_url: str = "http://localhost:6333",
        qdrant_collection: str = "sars_cov_news",
        embedding_model: str = "models/embedding-001",
        sql_model: str = "gemini-2.5-flash",
        summarization_model: str = "gemini-2.5-flash",
        final_model: str = "gemini-2.5-flash",
    ) -> None:
        """
        :param sqlite_path: caminho para o arquivo .db do SQLite
        :param qdrant_url: URL do servidor Qdrant
        :param qdrant_collection: nome da coleção onde estão as notícias
        :param embedding_model: modelo de embedding do Gemini
        :param sql_model: modelo de LLM para tradução NL→SQL
        :param summarization_model: modelo de LLM para resumo de resultados
        :param final_model: modelo de LLM para montar a resposta final
        """
        self.sqlite_path = sqlite_path
        self.qdrant = QdrantClient(url=qdrant_url)
        self.qdrant_collection = qdrant_collection
        self.embedding_model = embedding_model
        self.sql_model = sql_model
        self.summarization_model = summarization_model
        self.final_model = final_model

        genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

    def _execute_sql(self, sql: str) -> list[tuple]:
        """Executa a query SQL no banco SQLite e retorna todas as linhas."""
        with sqlite3.connect(self.sqlite_path) as conn:
            cur = conn.cursor()
            cur.execute(sql)
            return cur.fetchall()

    def _llm_chat(self, model: str, messages: list[dict]) -> str:
        """Chama o endpoint de chat do Gemini."""
        return (
            genai.GenerativeModel(model)
            .generate_content(
                "\n".join([f"{m['role']}: {m['content']}" for m in messages])
            )
            .text.strip()
        )

    def _natural_to_sql(self, question: str) -> str:
        """Traduz a pergunta em linguagem natural para uma consulta SQL válida."""
        return self._llm_chat(
            self.sql_model,
            [
                {
                    "role": "system",
                    "content": "You are a SQL query generator for SQLite.",
                },
                {
                    "role": "user",
                    "content": f"Translate this question into SQL:\n{question}",
                },
            ],
        )

    def _summarize(self, question: str, results: list[tuple]) -> str:
        """Gera um pequeno resumo explicativo dos resultados obtidos."""
        return self._llm_chat(
            self.summarization_model,
            [
                {"role": "system", "content": "You summarize table results."},
                {
                    "role": "user",
                    "content": (
                        "Question:\n"
                        + question
                        + "\nResults (list of tuples):\n"
                        + repr(results)
                        + "\nWrite a short paragraph explaining what those data shows."
                    ),
                },
            ],
        )

    def _get_embedding(self, text: str) -> list[float]:
        """Gera embedding para um texto usando a API do Gemini."""
        return genai.embed_content(model=self.embedding_model, content=text)[
            "embedding"
        ]

    def _search_news(self, embedding: list[float], limit: int = 5) -> list[dict]:
        """
        Busca notícias relacionadas no Qdrant, usando similaridade de coseno.
        Retorna lista de hits (id, payload).
        """
        return [
            {"id": hit.id, "score": hit.score, "payload": hit.payload}
            for hit in self.qdrant.search(
                collection_name=self.qdrant_collection,
                query_vector=embedding,
                limit=limit,
                with_payload=True,
            )
        ]

    def ask(self, question: str) -> str:
        """Ciclo principal: recebe pergunta, executa SQL, resume, busca notícias e devolve resposta final."""
        sql = self._natural_to_sql(question)
        summary = self._summarize(question, self._execute_sql(sql))
        return self._llm_chat(
            self.final_model,
            [
                {
                    "role": "system",
                    "content": "You are an agent that answers questions based on tabular data and related news.",
                },
                {
                    "role": "user",
                    "content": f"Question: {question}\n\n"
                    f"SQL generated:\n{sql}\n\n"
                    f"Data Summary:\n{summary}\n\n"
                    f"Related news (id and title):\n"
                    + "\n".join(
                        f"- {hit['id']}: {hit['payload'].get('title', '(sem título)')}"
                        for hit in self._search_news(self._get_embedding(summary))
                    ),
                },
            ],
        )

In [ ]:
agent = AIQueryAgent(sqlite_path="../data/interim/srag_2019_2024.db")
print(agent.ask("what was the month with more cases of sars in 2019"))